In [1]:
import sys
import dgl
import dgl.function as fn
import os
import multiprocessing as mp
from tqdm import tqdm
import pdb
import numpy as np
import torch
import torch.nn as nn
import logging
from utils.parser import parse_args
from utils.dataloader import Dataloader
from utils.utils import config, construct_negative_graph, choose_model, load_mf_model, NegativeGraph
from utils.tester import Tester
from models.sampler import NegativeSampler
import wandb
import math

In [34]:
numbers=[8,4,2,2,2,2,1,1]
square_sum=math.sqrt(sum(i**2 for i in numbers))
numbers = [x / square_sum for x in numbers]

ma=1.2*max(numbers)
mi=0.3*min(numbers)

result=[(x-mi) /(ma-mi) for x in numbers]


result = [x / sum(result) for x in result]

In [64]:
numbers=[8,4,2,2,2,2,1,1]
square_sum=math.sqrt(sum(i**2 for i in numbers))
numbers = [math.exp(x/square_sum) for x in numbers]
ma=max(numbers)
mi=0.2*min(numbers)
result=[(x-mi) /(ma-mi) for x in numbers]
result = [x / sum(result) for x in result]
result=[int(x*20) for x in result]


In [65]:
result

[4, 2, 2, 2, 2, 2, 1, 1]

In [3]:
def pairwise_cosine_similarity(x, y, zero_diagonal: bool = False):
    r"""
    Calculates the pairwise cosine similarity matrix

    Args:
        x: tensor of shape ``(batch_size, M, d)``
        y: tensor of shape ``(batch_size, N, d)``
        zero_diagonal: determines if the diagonal of the distance matrix should be set to zero

    Returns:
        A tensor of shape ``(batch_size, M, N)``
    """
    x_norm = torch.linalg.norm(x, dim=2, keepdim=True)
    y_norm = torch.linalg.norm(y, dim=2, keepdim=True)
    distance = torch.matmul(torch.div(x, x_norm), torch.div(y, y_norm).permute(0, 2, 1))
    

    return distance

In [4]:
x=torch.randn(2,4,2)
y=x
z=pairwise_cosine_similarity(x,y)

In [6]:
print(z)
print(z.mean(dim=2).unsqueeze(dim=2))

tensor([[[ 1.0000,  0.9999,  0.7630,  0.6927],
         [ 0.9999,  1.0000,  0.7550,  0.7015],
         [ 0.7630,  0.7550,  1.0000,  0.0624],
         [ 0.6927,  0.7015,  0.0624,  1.0000]],

        [[ 1.0000, -0.8350,  0.9246, -0.5884],
         [-0.8350,  1.0000, -0.5625,  0.9362],
         [ 0.9246, -0.5625,  1.0000, -0.2361],
         [-0.5884,  0.9362, -0.2361,  1.0000]]])
tensor([[[0.8639],
         [0.8641],
         [0.6451],
         [0.6142]],

        [[0.1253],
         [0.1347],
         [0.2815],
         [0.2779]]])
